<a href="https://colab.research.google.com/github/singwang-cn/The_first_pattern_recognition/blob/master/ex8_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from sklearn import svm

In [ ]:
data_path = '/content/drive/My Drive/Dataset/diabetes.csv'
pima = pd.read_csv(data_path)
pima_label = np.array(pima['Outcome'].tolist())
pima_nolabel = pima.drop(columns=['Outcome'])
pima_data = ((pima_nolabel-pima_nolabel.mean())/pima_nolabel.std()).values

In [3]:
class support_vector_machine():
  def __init__(self, kernel='rbf', sigma=4, c=10):
    self.kernel = kernel
    self.sigma = sigma
    self.c = c
    self.w = None
    self.b = None
  def fit(self, X):
    print(1)
  def recognize(self, x):
    result = np.dot(self.w, x) + b
    if result > 0:
      return 1
    if result < 0:
      return -1
  def rbf_kernel(self, u, v):
    norm = np.linalg.norm(u-v, ord=2)
    return np.exp(-1/(2*self.sigma)*(norm**2))
  def rbf_nonlinear(self, x):
    n = x.shape[0]-1
    #for i in range(n):
    

In [6]:
x1 = np.array([3, 4])
x2 = np.array([0, 0])
svm = support_vector_machine()
svm.rbf_kernel(x2, x1)

0.04393693362340742

In [44]:
H = np.array([[1, 1],
              [1, 0]])
t = np.array([1, 1])
c = 10
alpha = np.array([3, 4])

In [49]:
def obj_func(alpha):
  return 1/2*np.dot(np.dot(alpha, H), alpha)-np.sum(alpha)
def nonnega(alpha):
  return alpha[0]
def under_c(alpha):
  return c-alpha
def cons_eq(alpha):
  return np.dot(alpha, t)
cons = (
    {'type': 'ineq', 'fun': nonnega},
    {'type': 'ineq', 'fun': under_c},
    {'type': 'eq', 'fun': cons_eq}
)

In [50]:
minimize(obj_func, x0=alpha, constraints=cons, method="SLSQP")

     fun: -1.4199496293978212e-29
     jac: array([-0.99999999, -1.        ])
 message: 'Optimization terminated successfully.'
    nfev: 8
     nit: 2
    njev: 2
  status: 0
 success: True
       x: array([ 5.32907052e-15, -5.32907052e-15])

array([ True,  True])